In [175]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [176]:
service = Service("/usr/bin/chromedriver")  # Update path if different
driver = webdriver.Chrome(service=service)

driver.get("https://www.imdb.com/chart/top/") 
info=driver.find_element(By.XPATH,f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/div[1]/div/div/button[1]")
info.click()

In [177]:
Movies=pd.DataFrame(0,index=range(1,101),columns=['Movie Title','Image','Release Date','Duration Length','film Rating','Rating','Country of Origin','Directors','Stars','Summary or Plot'])
links=[None]*100

In [178]:
for i in range(1,101):
        info=driver.find_element(By.XPATH,f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[2]/div[1]/a")
        links[i-1]=info.get_attribute('href')
        
        Directors=""
        k=7
        j=2
        while True:
                Director= driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[2]/span/span[{j}]/a")
                Directors=Directors+Director.text+", "
                try:
                        Director=driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[2]/span/span[{j+1}]/a")
                        j=j+1
                except:
                        Movies.iloc[(i-1),k]=Directors[:-2]
                        if k==7 :
                                k=8
                        else:
                                break
                        j=j+2
                        Directors=""
        
        title=driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[2]/div[1]/a/h3")
        Movies.iloc[(i-1),0]=title.text

        image=driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[1]/div/div[2]/img")
        Movies.iloc[(i-1),1]=image.get_attribute('src')

        for j in range(2,4):
                try:
                        elem= driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[2]/div[2]/span[{j}]")
                        Movies.iloc[(i-1),(j+1)]=elem.text
                except:
                        continue

        Rating=driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[2]/span/div/span/span[1]")
        number=driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[1]/div[2]/span/div/span/span[2]")
        Movies.iloc[(i-1),5]=Rating.text+number.text

        plot= driver.find_element(By.XPATH, f"/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div/div/div/div[2]/div/div")
        Movies.iloc[(i-1),9]=plot.text
        

/tmp/ipykernel_5409/3767344847.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Frank Darabont' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Movies.iloc[(i-1),k]=Directors[:-2]
/tmp/ipykernel_5409/3767344847.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Tim Robbins, Morgan Freeman, Bob Gunton' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Movies.iloc[(i-1),k]=Directors[:-2]
/tmp/ipykernel_5409/3767344847.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1. The Shawshank Redemption' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Movies.iloc[(i-1),0]=title.text
/tmp/ipykernel_5409/3767344847.py:27: FutureWarning: Setting an item of inc

In [179]:
for i in range(0,100):
        driver.get(links[i])
        for k in range (10,14):
                try:
                        Release_Date=driver.find_element(By.XPATH,f"/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[{k}]/div[2]/ul/li[1]/div/ul/li/a")
                        Movies.iloc[i,2]=Release_Date.text

                        Countries=""
                        for j in range(1,10):
                                try:
                                        Country=driver.find_element(By.XPATH,f"/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[{k}]/div[2]/ul/li[2]/div/ul/li[{j}]/a")
                                        Countries=Countries+Country.text+", "
                                except:
                                        Movies.iloc[i,6]=Countries[:-2]
                                        break
                except:
                        continue

/tmp/ipykernel_5409/1805743048.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'October 14, 1994 (United States)' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Movies.iloc[i,2]=Release_Date.text
/tmp/ipykernel_5409/1805743048.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'United States' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Movies.iloc[i,6]=Countries[:-2]


In [180]:
driver.quit()
Movies.to_json('movies.json',orient='records',indent=4)

Error terminating service process.
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/selenium/webdriver/common/service.py", line 170, in _terminate_process
    self.process.terminate()
  File "/usr/lib/python3.12/subprocess.py", line 2211, in terminate
    self.send_signal(signal.SIGTERM)
  File "/usr/lib/python3.12/subprocess.py", line 2203, in send_signal
    os.kill(self.pid, sig)
PermissionError: [Errno 13] Permission denied
